In [ ]:
## import serial
import time
from matplotlib import pyplot as plt
import csv
import os
import datetime
import random
import numpy as np
import serial.tools.list_ports
from matplotlib.ticker import NullFormatter


def seleccionar_puerto():
    puertos_disponibles = [puerto.device for puerto in serial.tools.list_ports.comports()]

    if not puertos_disponibles:
        print("No se encontraron puertos seriales disponibles.")
        return None

    print("SELECCIONE PUERTO SERIAL:")
    for indice, puerto in enumerate(puertos_disponibles, start=1):
        print(f"{indice}. {puerto}")

    seleccion = 0
    while True:
        try:
            seleccion = int(input("Seleccione el número del puerto a usar: "))
            print("------------------------------------------------")
            if 1 <= seleccion <= len(puertos_disponibles):
                break
            else:
                print("Seleccion incorrecta. Intente de nuevo.")
        except ValueError:
            print("Entrada inválida. Por favor, ingrese un número.")

    puerto_seleccionado = puertos_disponibles[seleccion - 1]
    return puerto_seleccionado


puerto = seleccionar_puerto()



fecha = datetime.datetime.now().strftime("%d-%m-%Y_%H;%M;%S")
#ruta_carpeta = r'C:\Users\ManuelMR\Documents\electronervios'
ruta_carpeta = r'C:\Users\Dr. Nicolás Martínez\Documents\electro nervios'



#dacPin = 25; adcPin = 34;  

vector = []
def main():
    arduino = serial.Serial(puerto, 115200, timeout=1) #windows
    time.sleep(1) # Espera a que Arduino se inicialice
    arduino.flushInput()
    print("MENU PRINCIPAL")  
           ¿configurar cuanto tiempo de grabación?
             ¿cada cuanto tiempo se realiza la lectura análogo?
             ¿como se guardan los datos y donde?
    print("1. Seleccione tiempo de muestreo")
    print("2. Salir")
    while True:
        t_muestreo = input("Selecciona una opción, o presiona '3' para salir: ")
        
        if t_muestreo == '1':
            while True:
                print("1. Señal Cuadrada")
                print("2. Señal Sinoidal")
                print("3. Salir")
                señal = input("Selecciona un tipo de señal, o presione '3' para salir: ")
               
                if señal == '1':
                    print("Has elegido señal cuadrada")
                    pulso = input("Seleccione un ancho de pulso en (uS):")
                    print(type(pulso))
                    while True:
                        amplitud = input("Seleccione una amplitud en mV:")
                        if int(amplitud) > 3200 or int(amplitud)<=0:
                            print("Error, voltaje fuera del rango")
                        else:
                            break
                    Nveces = input("Seleccione la cantidad de pulsos:")
                    intervalo = input("Seleccione un intervalo en (uS):")
                    intervalo_adc = input("Seleccione un intervalo de tiempo (mS) entre lecturas:")
                   
                    while True:
                        Nveces_adc = input("Seleccione la cantidad de lecturas:")
                        if int(Nveces_adc)>1000:
                            print("Error, lectura fuera de rango")
                        else:
                            break
                       
                    while True:
                        iniciar = str(input("Quiere iniciar el programa? (si / no):"))
                        if iniciar =="si":
                            print("iniciando el programa...")
                            instruccion = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo)+','+str(intervalo_adc)+','+str(Nveces_adc)+','+str(iniciar)+'\r'
                            print("Enviando a Arduino: ", instruccion)
                            time.sleep(1)
                            arduino.write(instruccion.encode('utf-8'))
                            time.sleep(1)
                            response = arduino.readline().decode().strip()
                            print("Respuesta de Arduino:", response)
                            contador = 0

                            while True:
                                if arduino.in_waiting > 0:
                                    serial_data = arduino.readline().decode().rstrip()
                                    vector.append(int(serial_data))                    

                                    contador= contador+1
                                    if contador == int(Nveces_adc):
                                        break
                                    else:
                                        if len(vector)==(int(Nveces_adc)-1):
                                            #print(vector)
                                            vector_m = [x * 3.3 for x in vector]
                                            vector_voltaje = [round(x / 255, 2) for x in vector_m]
                                            print(vector_voltaje)
                                            tiempo = np.arange(0, len(vector), 1)
                                            #tiempo = np.arange(0, len(vector) * 3, 3)
                                            print("MENU PLOTEO")
                                            print("1. Plot datos brutos")
                                            print("2. Plot datos en voltaje")
                                            print("3. Plot ambos")
                                            plot_archivo = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo)+','+str(intervalo_adc)+','+str(Nveces_adc)+'('+str(fecha)+').png'
                                            plot_archivo2 = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo)+','+str(intervalo_adc)+','+str(Nveces_adc)+'('+str(fecha)+')_2.png'
                                            nombre_archivo = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo)+','+str(intervalo_adc)+','+str(Nveces_adc)+'('+str(fecha)+').csv'
                                            nombre_carpeta = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo)+','+str(intervalo_adc)+','+str(Nveces_adc)+'('+str(fecha)+')'
                                            ruta_carpeta2 = os.path.join(ruta_carpeta, nombre_carpeta)
                                            os.makedirs(ruta_carpeta2)
                                            ruta_completa_csv = os.path.join(ruta_carpeta2, nombre_archivo)
                                            ruta_completa_plot = os.path.join(ruta_carpeta2, plot_archivo)
                                            ruta_completa_plot2 = os.path.join(ruta_carpeta2, plot_archivo2)
                                            plotteo = str(input("Que quieres plotear?:"))
                                            if plotteo == '1':
                                                fig, ax = plt.subplots()
                                                ax.plot(tiempo,vector)
                                                ax.set(xlabel='Tiempo (ms)', ylabel='Valor Bruto', title='Gráfico de lectura ADC bruto')
                                                ax.grid(True, linestyle='--', linewidth=0.5)
                                                ax.minorticks_on()
                                                ax.grid(True, which='minor', color='#999999', linestyle='--', alpha=0.5)
                                                plt.title("Pulsos:"+str(Nveces)+" | "+str(intervalo_adc)+"ms entre lecturas | cantidad de lecturas: "+str(Nveces_adc)+"veces",fontsize=10)
                                                plt.suptitle("Lecturas ADC (Valor Bruto [0-255])")
                                                plt.savefig(ruta_completa_plot, dpi=300)
                                            if plotteo == '2':
                                                fig, ax = plt.subplots()
                                                ax.plot(tiempo,vector_voltaje)                                              
                                                ax.set(xlabel='Tiempo (ms)', ylabel='Voltaje', title='Gráfico de lectura ADC en voltaje')
                                                ax.grid(True, linestyle='--', linewidth=0.5)
                                                ax.minorticks_on()
                                                ax.grid(True, which='minor', color='#999999', linestyle='--', alpha=0.5)
                                                plt.title("Pulsos:"+str(Nveces)+" | "+str(intervalo_adc)+"ms entre lecturas | cantidad de lecturas: "+str(Nveces_adc)+"veces",fontsize=10)
                                                plt.suptitle("Lecturas ADC (Voltaje [0-3.3])")
                                                plt.savefig(ruta_completa_plot, dpi=300)
                                            if plotteo == '3':
                                                plt.figure(1)
                                                plt.plot(tiempo,vector)
                                                plt.title('Gráfico de lectura ADC bruto')
                                                plt.xlabel('Tiempo (ms)')
                                                plt.ylabel('Valor Bruto')
                                                plt.title("Pulsos:"+str(Nveces)+" | "+str(intervalo_adc)+"ms entre lecturas | cantidad de lecturas: "+str(Nveces_adc)+"veces",fontsize=10)
                                                plt.suptitle("Lecturas ADC (Valor Bruto [0-255])")
                                                plt.savefig(ruta_completa_plot, dpi=300)
                                                #################
                                                plt.figure(2)
                                                plt.plot(tiempo,vector_voltaje)
                                                plt.title('Gráfico de lectura ADC en voltaje')
                                                plt.xlabel('Tiempo (ms)')
                                                plt.ylabel('Voltaje')
                                                plt.title("Pulsos:"+str(Nveces)+" | "+str(intervalo_adc)+"ms entre lecturas | cantidad de lecturas: "+str(Nveces_adc)+"veces",fontsize=10)
                                                plt.suptitle("Lecturas ADC (Voltaje [0-3.3])")
                                                plt.savefig(ruta_completa_plot2, dpi=300)

                                            plt.show()


                                            if plotteo == '1':
                                                with open(ruta_completa_csv, 'w', newline='') as archivo_csv:
                                                    escritor_csv = csv.writer(archivo_csv)
                                                    escritor_csv.writerow(['Lecturas ADC (Valor Bruto [0-255])'])
                                                    for fila in vector:
                                                        escritor_csv.writerow([fila])
                                            if plotteo == '2':
                                                with open(ruta_completa_csv, 'w', newline='') as archivo_csv:
                                                    escritor_csv = csv.writer(archivo_csv)
                                                    escritor_csv.writerow(['Lecturas ADC (Voltaje [0-3.3])'])
                                                    for fila in vector_voltaje:
                                                        escritor_csv.writerow([fila])

                                            if plotteo == '3':
                                                with open(ruta_completa_csv, 'w', newline='') as archivo_csv:
                                                    escritor_csv = csv.writer(archivo_csv)
                                                    escritor_csv.writerow(['Lecturas ADC (Valor Bruto [0-255])'])
                                                    for fila in vector:
                                                        escritor_csv.writerow([fila])
                                               
                                                with open(ruta_completa_csv, 'w', newline='') as archivo_csv2:
                                                    escritor_csv2 = csv.writer(archivo_csv2)
                                                    escritor_csv2.writerow(['Lecturas ADC (Voltaje [0-3.3])'])
                                                    for fila in vector_voltaje:
                                                        escritor_csv2.writerow([fila])
                                            print(f'Se han guardado los archivos PNG y CSV correctamente en {ruta_completa_csv}')
                                            time.sleep(1)
                                            print("CONTINUAR?")
                                            print("1. Si")
                                            print("2. No")
                                            reiniciar = input("Seleccione una t_muestreo: ")

                                            
                                            if reiniciar=="1":
                                                arduino.close()
                                                time.sleep(2)
                                                main()
                                            if reiniciar =="2":
                                                print("Adios")
                                                arduino.close()
                                                time.sleep(2)

                           
                        elif iniciar =="no":
                            print("Saliendo del programa...")
                            arduino.close()
                            break
                        else:
                            print("Error")
    
                if señal == '2':
                    print("has elegido señal senoidal")
                    while True:
                        try:
                            pulso = int(input("Seleccione el periodo(us) de la onda senoidal:"))
                            pulso = str(pulso)
                            break
                        except ValueError:
                            print("Error: Debes ingresar un número.")
                    
                    while True:
                        amplitud = input("Seleccione una amplitud en mV (peak-to-peak):")
                        if int(amplitud) > 3000:
                            print("Error, voltaje fuera del rango")
                        else:
                            break
                    Nveces = input("Seleccione la cantidad de ondas senoidales:")
                    intervalo_adc = input("Seleccione un intervalo de tiempo (mS) entre lecturas:")
                    while True:
                        Nveces_adc = input("Seleccione la cantidad de lecturas:")
                        if int(Nveces_adc)>1000:
    
                            print("Error, lectura fuera de rango")
                        else:
                            break
                    while True:
                        iniciar = str(input("Quiere iniciar el programa #2? (si / no):"))
                        if iniciar =="si":
                            print("iniciando del programa...")
                            instruccion = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+',1,'+str(intervalo_adc)+','+str(Nveces_adc)+','+str(iniciar)+'\r'
                            print(instruccion)
                            time.sleep(2)
                            arduino.write(instruccion.encode('utf-8'))
                            time.sleep(1)
                            response = arduino.readline().decode().strip()
                            print("Respuesta de Arduino:", response)
                            contador = 0
                            try:
                                while True:
                                    if arduino.in_waiting > 0:
                                        serial_data = arduino.readline().decode().rstrip()
                                        vector.append(int(serial_data))                    
                                        #print(vector)
                                        
                                        contador= contador+1
                                        if contador == int(Nveces_adc):
                                            break
                                        else:
                                            if len(vector)==(int(Nveces_adc)-1):
                                                vector_m = [x * 3.3 for x in vector]
                                                vector_voltaje = [round(x / 255, 2) for x in vector_m]
                                                print(vector_voltaje)
                                                #print(vector)
                                                #tiempo = np.arange(0, len(vector),1)
                                                tiempo = np.arange(0, len(vector) * 0.7, 0.7)
                                                print("MENU PLOTEO")
                                                print("1. Plot datos brutos")
                                                print("2. Plot datos en voltaje")
                                                print("3. Plot ambos")
                                                plot_archivo = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo_adc)+','+str(Nveces_adc)+'('+str(fecha)+').png'
                                                plot_archivo2 = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo_adc)+','+str(Nveces_adc)+'('+str(fecha)+')_2.png'
                                                nombre_archivo = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo_adc)+','+str(Nveces_adc)+'('+str(fecha)+').csv'
                                                nombre_carpeta = str(t_muestreo)+','+str(señal)+','+str(pulso)+','+str(amplitud)+','+str(Nveces)+','+str(intervalo_adc)+','+str(Nveces_adc)+'('+str(fecha)+')'
                                                ruta_carpeta2 = os.path.join(ruta_carpeta, nombre_carpeta)
                                                os.makedirs(ruta_carpeta2)
                                                ruta_completa_csv = os.path.join(ruta_carpeta2, nombre_archivo)
                                                ruta_completa_plot = os.path.join(ruta_carpeta2, plot_archivo)
                                                ruta_completa_plot2 = os.path.join(ruta_carpeta2, plot_archivo2)
                                                plotteo = str(input("Que quieres plotear?:"))
                                                if plotteo == '1':
                                                    fig, ax = plt.subplots()
                                                    ax.plot(tiempo,vector)
                                                    ax.set(xlabel='Tiempo (ms)', ylabel='Valor Bruto', title='Gráfico de lectura ADC bruto')
                                                    ax.grid(True, linestyle='--', linewidth=0.5)
                                                    ax.minorticks_on()
                                                    ax.grid(True, which='minor', color='#999999', linestyle='--', alpha=0.5)
                                                    plt.title("Pulsos:"+str(Nveces)+" | "+str(intervalo_adc)+"ms entre lecturas | cantidad de lecturas: "+str(Nveces_adc)+"veces",fontsize=10)
                                                    plt.suptitle("Lecturas ADC (Valor Bruto [0-255])")
                                                    plt.savefig(ruta_completa_plot, dpi=300)
                                                if plotteo == '2':
                                                    fig, ax = plt.subplots()
                                                    ax.plot(tiempo,vector_voltaje)                                              
                                                    ax.set(xlabel='Tiempo (ms)', ylabel='Voltaje', title='Gráfico de lectura ADC en voltaje')
                                                    ax.grid(True, linestyle='--', linewidth=0.5)
                                                    ax.minorticks_on()
                                                    ax.grid(True, which='minor', color='#999999', linestyle='--', alpha=0.5)
                                                    plt.title("Pulsos:"+str(Nveces)+" | "+str(intervalo_adc)+"ms entre lecturas | cantidad de lecturas: "+str(Nveces_adc)+"veces",fontsize=10)
                                                    plt.suptitle("Lecturas ADC (Voltaje [0-3.3])")
                                                    plt.savefig(ruta_completa_plot, dpi=300)
                                                if plotteo == '3':
                                                    plt.figure(1)
                                                    plt.plot(tiempo,vector)
                                                    plt.title('Gráfico de lectura ADC bruto')
                                                    plt.xlabel('Tiempo (ms)')
                                                    plt.ylabel('Valor Bruto')
                                                    plt.title("Pulsos:"+str(Nveces)+" | "+str(intervalo_adc)+"ms entre lecturas | cantidad de lecturas: "+str(Nveces_adc)+"veces",fontsize=10)
                                                    plt.suptitle("Lecturas ADC (Valor Bruto [0-255])")
                                                    plt.savefig(ruta_completa_plot, dpi=300)
                                                    #################
                                                    plt.figure(2)
                                                    plt.plot(tiempo,vector_voltaje)
                                                    plt.title('Gráfico de lectura ADC en voltaje')
                                                    plt.xlabel('Tiempo (ms)')
                                                    plt.ylabel('Voltaje')
                                                    plt.title("Pulsos:"+str(Nveces)+" | "+str(intervalo_adc)+"ms entre lecturas | cantidad de lecturas: "+str(Nveces_adc)+"veces",fontsize=10)
                                                    plt.suptitle("Lecturas ADC (Voltaje [0-3.3])")
                                                    plt.savefig(ruta_completa_plot2, dpi=300)
    
                                                plt.show()
    
    
                                                if plotteo == '1':
                                                    with open(ruta_completa_csv, 'w', newline='') as archivo_csv:
                                                        escritor_csv = csv.writer(archivo_csv)
                                                        escritor_csv.writerow(['Lecturas ADC (Valor Bruto [0-255])'])
                                                        for fila in vector:
                                                            escritor_csv.writerow([fila])
                                                if plotteo == '2':
                                                    with open(ruta_completa_csv, 'w', newline='') as archivo_csv:
                                                        escritor_csv = csv.writer(archivo_csv)
                                                        escritor_csv.writerow(['Lecturas ADC (Voltaje [0-3.3])'])
                                                        for fila in vector_voltaje:
                                                            escritor_csv.writerow([fila])
    
                                                if plotteo == '3':
                                                    with open(ruta_completa_csv, 'w', newline='') as archivo_csv:
                                                        escritor_csv = csv.writer(archivo_csv)
                                                        escritor_csv.writerow(['Lecturas ADC (Valor Bruto [0-255])'])
                                                        for fila in vector:
                                                            escritor_csv.writerow([fila])
                                                   
                                                    with open(ruta_completa_csv, 'w', newline='') as archivo_csv2:
                                                        escritor_csv2 = csv.writer(archivo_csv2)
                                                        escritor_csv2.writerow(['Lecturas ADC (Voltaje [0-3.3])'])
                                                        for fila in vector_voltaje:
                                                            escritor_csv2.writerow([fila])
                                                print(f'Se han guardado los archivos PNG y CSV correctamente en {ruta_completa_csv}')
    
                            except KeyboardInterrupt:
                                arduino.close()  
                            arduino.close()
                            print("--------------------------------------------")
                            #main()
                        
                        elif iniciar =="no":
                            print("Saliendo del programa Señal File ...")
                            break
                        else:
                            print("Error")
    
                elif señal.lower() == '3':
                    print("Saliendo del programa...")
                    arduino.close()
                    break
                else:
                    break
                    print("Opción inválida. Por favor, selecciona una opción válida.")
                    
        elif t_muestreo == '2':
            print("Finalizado...")
            arduino.close()
            time.sleep(2)
            break
        else:
            print("Opción inválidaaa. Por favor, selecciona una opción válida.")
#
if __name__ == "__main__":
    main()